# Análise exploratória Airbnb (Rio de Janeiro)
**Autor:** Douglas Trajano

Este notebook irá reunir a análise exploratória feita nos dados do Airbnb (Rio de Janeiro).

O objetivo deste projeto é desenvolver um classificador para predizer o `room_type` de cada anúncio no Airbnb para o Rio de Janeiro. Por isso, muitas visualizações serão focadas em entender o comportamento dos dados perante essa variável.

A estrutura completa do projeto pode ser vista [aqui](https://github.com/DougTrajano/ds_airbnb_rio).

## / imports

In [77]:
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import utilis_script

# graphs
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## / load dataset

In [4]:
df_listings = utilis_script.get_data(origin="listings")
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33715 entries, 0 to 33714
Columns: 106 entries, id to reviews_per_month
dtypes: float64(24), int64(21), object(61)
memory usage: 27.3+ MB


## / Pandas Profiling

Eu gosto de gerar o ProfileReport através da biblioteca Pandas Profiling, pois é possível ter uma visão inicial do dataset muito mais rápido e útil para evoluirmos na nossa análise exploratória.

In [17]:
from IPython.core.display import HTML

profile = pdp.ProfileReport(df_listings, title='Airbnb RJ - listings.csv')
profile.to_file(output_file="airbnb_data_report.html")

HTML('<a href="airbnb_data_report.html" target="_blank">airbnb_data_report.html</a>')

## / preprocessing

Vamos fazer alguns preprocessamentos identificados através do Pandas Profiling.

Existem duas colunas que foram identificadas como **categorical** e que iremos convertê-las em **datetime**. São elas: `"calendar_last_scraped", "last_scraped"`.

Existem muitas colunas com apenas um valor, com todos os registros nulos ou grande parte dos registros nulos. Isso provavelmente não agregará ao nosso modelo. Como algumas dessas colunas são mais técnicas, por exemplo: `scrape_id` que deve representar o id de extração interno do airbnb, não vou analisar isto neste momento, o objetivo aqui é entender os dados na visão de negócio.

In [34]:
datetime_cols = ["calendar_last_scraped", "last_scraped"]

for col in datetime_cols:
    df_listings[col] = pd.to_datetime(df_listings[col])

## / Exploratory analysis

Agora vamos entender melhor os dados visualizando alguns gráficos.

### / room_type distribution

In [79]:
df_temp = df_listings["room_type"].value_counts()

fig = {
  "data": [
    {
      "values": df_temp.values,
      "labels": df_temp.index,
      "domain": {"x": [0, .48]},
      "hole": .7,
      "type": "pie"
    },
    
    ],
  "layout": {
        "title":"room_type distribution",
    }
}

fig = go.Figure(fig)
fig.update_layout(template="seaborn")
iplot(fig)

In [59]:
room_types = df_listings["room_type"].unique().tolist()

df_accommodates = df_listings[["room_type", "accommodates"]]

for i in range(len(df_accommodates)):
    if df_accommodates["accommodates"][i] > 10:
        df_accommodates["accommodates"][i] = "more than 10"

In [71]:
data_vis = []

for room_type in room_types:
    df_temp = df_accommodates[df_accommodates["room_type"] == room_type]
    y_values = df_temp.groupby("accommodates").count()["room_type"].tolist()
    temp = go.Bar(name=room_type, x=df_accommodates["accommodates"].unique(), y=y_values)
    data_vis.append(temp)
    
fig = go.Figure(data=data_vis)
fig.update_layout(barmode='group')
fig.show()